# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/ilenapeng/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/9x/ttxn_gwj0592mr6h8s5y29980000gn/T/ipykernel_85933/1450958847.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

In [3]:
# select cosmetologist
driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[3]/td/select/option[10]').click()

In [4]:
# search Nguyen
driver.find_element(By.ID, 'pht_lnm').send_keys('Nguyen')

In [5]:
# hit search
driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't actual tabular data!

> You can use either Selenium by itself or Selenium+BeautifulSoup to scrape the results page. The choice is up to you!

### Loop through each result and print the entire row

Okay wait, maybe not, i's a heck of a lot of rows. Use `[:10]` to only do the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

> *Tip: If you're using Selenium, `By.TAG_NAME` is used if you don't have a class or ID. If you're using BeautifulSoup, just do your normal thing.*

In [6]:
for cell in driver.find_elements(By.TAG_NAME, 'tr')[:10]:
    print(cell.text)

Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877


License #(s): 760420, 1620583

Complaint # COS20210009745 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,550. Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables pr

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – `driver.find_element` or `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
doc = BeautifulSoup(driver.page_source)

In [9]:
for row in doc.find_all('tr')[:10]:
    try:
        print(row.find('span').text)
    except:
        pass
    
# another way to do it without the try/except
# for row in doc.find_all('tr')[1:10]:
#     print(row.find('span').text)

# printing just the first name:
# doc.find_all('tr')[1].find('span').text

NGUYEN, THANH 
NGUYEN, LONG D
NGUYEN, LUCIE HUONG 
NGUYEN, CHINH 
NGUYEN, JIMMY 
NGUYEN, NAM 
NGUYEN, DUC 
NGUYEN, THU THAO THI 
NGUYEN, MINH NHU 


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: If you're using Selenium by itself, you can get the HTML of something by doing `.get_attribute('innerHTML')` – that way it'll look like BeautifulSoup when you print it. It might help you diagnose your issue!*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [10]:
for row in doc.find_all('tr')[:10]:
    try:
        print(row.find_all('td')[2].text)
    except:
        pass

# doc.find_all('tr')[1].find_all('td')[2].text

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client; Res

## Loop through each result, printing the complaint number

Output should look like this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [11]:
for row in doc.find_all('tr')[:10]:
    try:
        print(row.find_all('span')[-3].text)
    except:
        pass

# doc.find_all('tr')[1].find_all('span')[-3].text

COS20210004784
COS20210009745
COS20210011484
COS20210011721
COS20200007069
COS20210010530
COS20200007141
COS20200000839
COS20210009714


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [12]:
# Testing for each of the things in the datasets

# # Person's name
# doc.find_all('tr')[1].find('span').text

# # Violation description
# doc.find_all('tr')[1].find_all('td')[2].text

# # Violation number
# doc.find_all('tr')[1].find_all('span')[-3].text

# # License Numbers
# doc.find_all('tr')[1].find_all('span')[-5].text

# # Zip Code
# doc.find_all('tr')[1].find_all('span')[6].text

# # County
# doc.find_all('tr')[1].find_all('span')[4].text

# # City
# doc.find_all('tr')[1].find_all('span')[2].text

In [13]:
dataset = []

for row in doc.find_all('tr')[1:]:
    data = {}
    data['name'] = row.find('span').text
    data['violation'] = row.find_all('td')[2].text
    data['violation_num'] = row.find_all('span')[-3].text
    data['license_num'] = row.find_all('span')[-5].text
    data['zipcode'] = row.find_all('span')[6].text
    data['county'] = row.find_all('span')[4].text
    data['city'] = row.find_all('span')[2].text
    dataset.append(data)

# dataset

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [14]:
df = pd.DataFrame(dataset)
df.head()

,name,violation,violation_num,license_num,zipcode,county,city
0,"NGUYEN, THANH",Respondent failed to clean and sanitize whirlp...,COS20210004784,790672,75034,COLLIN,FRISCO
1,"NGUYEN, LONG D",Respondent failed to keep a record of the date...,COS20210009745,"760420, 1620583",76877,SAN SABA,SAN SABA
2,"NGUYEN, LUCIE HUONG",Respondent failed to keep a record of the date...,COS20210011484,"762626, 1811788",78801,UVALDE,UVALDE
3,"NGUYEN, CHINH",Respondent failed to follow whirlpool foot spa...,COS20210011721,777067,76502,BELL,TEMPLE
4,"NGUYEN, JIMMY",Respondent failed to clean and sanitize whirlp...,COS20200007069,796773,75088,DALLAS,ROWLETT


In [15]:
df.to_csv('output.csv', index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [16]:
pd.read_csv('output.csv')

,name,violation,violation_num,license_num,zipcode,county,city
0,"NGUYEN, THANH",Respondent failed to clean and sanitize whirlp...,COS20210004784,790672,75034,COLLIN,FRISCO
1,"NGUYEN, LONG D",Respondent failed to keep a record of the date...,COS20210009745,"760420, 1620583",76877,SAN SABA,SAN SABA
2,"NGUYEN, LUCIE HUONG",Respondent failed to keep a record of the date...,COS20210011484,"762626, 1811788",78801,UVALDE,UVALDE
3,"NGUYEN, CHINH",Respondent failed to follow whirlpool foot spa...,COS20210011721,777067,76502,BELL,TEMPLE
4,"NGUYEN, JIMMY",Respondent failed to clean and sanitize whirlp...,COS20200007069,796773,75088,DALLAS,ROWLETT
...,...,...,...,...,...,...,...
96,"NGUYEN, SON DANG",Respondent failed to clean and sanitize whirlp...,COS20190010635,"720091, 1195752",76119,TARRANT,FOREST HILL
97,"NGUYEN, TUYEN THANH",Respondent failed to clean and sanitize whirlp...,COS20190014566,"792178, 1468619, 1410422",76227,DENTON,AUBREY
98,"NGUYEN, STEVEN THAI",Respondent failed to clean and sanitize whirlp...,COS20190010869,779635,75110,NAVARRO,CORSICANA
99,"NGUYEN, CY VAN",Respondent operated a cosmetology salon withou...,COS20190007921,765590,77007,HARRIS,HOUSTON
